In [389]:
import pandas as pd
import numpy as np
import pickle
from Bio import SeqIO
from Bio import Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Remove recombinations from MSA

+ input: MSA (fa, xmfa), tree, predicted importations 
+ output 1: MSA without recombination
    + fasta
    + phylip
+ output 2: table of protein ids assigned to importations 


# Results for the test sets

## MSA lengths
+ rso-test (fjat) = 3,825,198 (xmfa: 7,666,198)
+ phy2 = 2,987,190 (xmfa: 6,086,190)
+ phy4 = 3,219,942
+ phy3 = 3,459,843 (xmfa: 6,947,843)

(for ASM359060v in phy2 dataset the summed length is 116,425)

## Summed recombinations length
+ rso = 92,925
+ phy2 = 290,218
+ phy4 = 58,063
+ phy3 = 120,259

(phy4 strain PSI07 is from Indonesia! very different)

In [390]:
# in

recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.importation_status.txt'
path2tree = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.labelled_tree.newick'
xm_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.xmfa"
fa_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.fasta"
accDB = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/MSA_ACC_DB_rso_test.pkl'
protDB = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/GENOME_PROTEIN_DB_rso_test.pkl'
# out
fa_msa_out = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat_no_RR.fasta"
ph_msa_out = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat_no_RR.phylip"
updated_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis2prot.csv'
mult_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_mult_genes.txt'
top_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_top_genes.txt'


# in
SAMPLE = 'phy4_test'
recombis = recombis.replace("rso_test", SAMPLE)
path2tree = path2tree.replace("rso_test", SAMPLE)
xm_msa_in = xm_msa_in.replace("rso_test", SAMPLE)
fa_msa_in = fa_msa_in.replace("rso_test", SAMPLE)
accDB = accDB.replace("rso_test", SAMPLE)
protDB = protDB.replace("rso_test", SAMPLE)
# out
fa_msa_in = fa_msa_in.replace("rso_test", SAMPLE)
ph_msa_out = ph_msa_out.replace("rso_test", SAMPLE)
updated_recombis = updated_recombis.replace("rso_test", SAMPLE)
mult_recombis = mult_recombis.replace("rso_test", SAMPLE)
top_recombis = top_recombis.replace("rso_test", SAMPLE)



# in
#recombis = snakemake.params["recombis"]
#path2tree = snakemake.params["tree"]
#xm_msa_in = snakemake.input["xm_msa_in"]
#fa_msa_in = snakemake.input["fa_msa_in"]
#accDB = snakemake.input["accDB"]
#protDB = snakemake.input["protDB"]
# out
#fa_msa_out = snakemake.output["fa_msa_out"]
#ph_msa_out = snakemake.output["ph_msa_out"]
#updated_recombis = snakemake.output["updated_recombis"]
#mult_recombis = snakemake.output["mult_recombis"]
#top_recombis = snakemake.output["top_recombis"]


# Replace the inner nodes with the leaves 

In [391]:
imp_stat = pd.read_csv(recombis, sep='\t')
#imp_stat

In [392]:
for tree in Phylo.parse(path2tree, 'newick'):
    parents = {}
    # iterate through inner nodes 
    for clade in tree.get_nonterminals(order="level"):
        parents[clade.name] = []
        # get leaves for each inner node
        for child in clade.get_terminals():
            parents[clade.name].append(child.name)
        #parents[clade.name] = ";".join(parents[clade.name])
parents

{'NODE_25': ['ASM28347v1',
  'ASM351538v',
  'ASM351552v',
  'ASM351532v',
  'ASM351542v',
  'ASM351548v',
  'ASM351518v',
  'ASM351526v',
  'ASM351514v',
  'ASM351550v',
  'ASM351544v',
  'ASM351516v',
  'ASM158613v'],
 'NODE_24': ['ASM28347v1', 'ASM351538v'],
 'NODE_23': ['ASM351552v',
  'ASM351532v',
  'ASM351542v',
  'ASM351548v',
  'ASM351518v',
  'ASM351526v',
  'ASM351514v',
  'ASM351550v',
  'ASM351544v',
  'ASM351516v',
  'ASM158613v'],
 'NODE_22': ['ASM351552v', 'ASM351532v'],
 'NODE_21': ['ASM351542v',
  'ASM351548v',
  'ASM351518v',
  'ASM351526v',
  'ASM351514v',
  'ASM351550v',
  'ASM351544v',
  'ASM351516v',
  'ASM158613v'],
 'NODE_20': ['ASM351548v',
  'ASM351518v',
  'ASM351526v',
  'ASM351514v',
  'ASM351550v',
  'ASM351544v',
  'ASM351516v',
  'ASM158613v'],
 'NODE_19': ['ASM351518v',
  'ASM351526v',
  'ASM351514v',
  'ASM351550v',
  'ASM351544v',
  'ASM351516v',
  'ASM158613v'],
 'NODE_14': ['ASM351518v', 'ASM351526v'],
 'NODE_18': ['ASM351514v',
  'ASM351550v',
  '

In [393]:
new_len = len(imp_stat)

for p in parents:

    is_node = imp_stat["Node"].str.startswith(p)
    num_child = len(parents[p]) 
    print(p,'has',len(imp_stat[is_node]),'recombinations and',num_child,'children')
    new_len = new_len + len(imp_stat[is_node]) * (num_child - 1)
    
    #imp_stat = imp_stat.append([imp_stat[is_node]]*(num_child-1), ignore_index=True)
    print()
    
    # for each child: duplicate the parent row
    for child in parents[p]:       
        node_imp = imp_stat[is_node].replace({p:child}) 
        imp_stat = imp_stat.append([node_imp], ignore_index=True)
        is_node = imp_stat["Node"].str.startswith(p)

# remove the remaining parent rows

imp_stat = imp_stat.loc[~imp_stat["Node"].str.contains("NODE_")]
print('DF length after parent nodes expantion =', len(imp_stat), '; Must be =', new_len)
assert(new_len == len(imp_stat))

NODE_25 has 0 recombinations and 13 children

NODE_24 has 0 recombinations and 2 children

NODE_23 has 10 recombinations and 11 children

NODE_22 has 0 recombinations and 2 children

NODE_21 has 1 recombinations and 9 children

NODE_20 has 0 recombinations and 8 children

NODE_19 has 18 recombinations and 7 children

NODE_14 has 83 recombinations and 2 children

NODE_18 has 7 recombinations and 5 children

NODE_17 has 0 recombinations and 4 children

NODE_16 has 0 recombinations and 3 children

NODE_15 has 0 recombinations and 2 children

DF length after parent nodes expantion = 569 ; Must be = 569


# Sort and sum the importations

In [394]:
imp_stat = imp_stat.sort_values(by=['Beg', 'End'])
#imp_stat

In [395]:
beg = imp_stat["Beg"].values.copy()
end = imp_stat["End"].values.copy()

In [396]:
ranges2del = []
for i in range(len(imp_stat)):
    ranges2del.extend(range(beg[i]-1, end[i]))
ranges2del = list(dict.fromkeys(ranges2del))
print('Summed recombination length:', len(ranges2del))

Summed recombination length: 58063


# Fasta vs. XMFA positions mapping list

* list of updated positions 
* index represent Fasta sequence positions 
* values represent XMFA sequence positions 

In [397]:
# parse the genes MSAs into vector of the alignments' lengths
aln_length = []
aln = open(xm_msa_in, 'r')

for l in aln:
    
    if l.startswith('>'):
        c = 0
    elif l.startswith('='):
        aln_length.append(c-1) # minus one char for the \n symbol
    else:
        c = c + len(l)
        
print('Found genes MSAs / LCBs:', len(aln_length))    
aln.close()

Found genes MSAs / LCBs: 3372


In [398]:
# insert 1000 count after each gene MSA
index_mapping = list(range(0,aln_length[0]))
sum_lengths = aln_length[0]

for i in range(1,len(aln_length)):  
    index_mapping.extend(range(sum_lengths + i*1000, sum_lengths + aln_length[i] + i*1000))
    sum_lengths = sum_lengths + aln_length[i]
    

In [399]:
print('Fasta length', sum(aln_length), '\nXMFA length', sum(aln_length)+(len(aln_length)-1)*1000)
# index = fasta file length
assert(sum(aln_length) == len(index_mapping))
# values = xmfa file length
assert(sum(aln_length)+(len(aln_length)-1)*1000 == index_mapping[-1]+1) 

Fasta length 3219942 
XMFA length 6590942


# Get the fasta position of each insertion from the xmfa position

+ use ranges to improve the running time of .index(i, start, end)

In [400]:
fa_beg = []
fa_end = []
previous_beg = 0
previous_end = 0
L=len(index_mapping)

for r_i in range(len(imp_stat)):
    
    # -1 for 0-start indexing
    r_beg = beg[r_i] - 1
    r_end = end[r_i] - 1
    #print(r_beg, r_end)
    
    # USE PREVIOUS_BEG FOR BOTH BECAUSE OF THE SORTING ISSUE
    r_beg_index = index_mapping.index(r_beg, previous_beg, L)#r_beg+1)
    r_end_index = index_mapping.index(r_end, previous_beg, L)#r_end+1)
    
    previous_beg = r_beg_index
    previous_end = r_end_index
    
    fa_beg.append(r_beg_index)
    fa_end.append(r_end_index)
    
    #print(r_i,"xmfa - fa:",r_beg_index, r_end_index)

imp_stat['FA Beg'] = fa_beg
imp_stat['FA End'] = fa_end

In [401]:
imp_stat['debug_len_must'] = imp_stat['End'] - imp_stat['Beg']
imp_stat["debug_len_is"] = imp_stat['FA End'] - imp_stat['FA Beg']
#assert(len(imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]) == 0)
#imp_stat
imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']].to_csv(mult_recombis, index=False)

# Delete the recombinations from the fasta file 

In [402]:
fasta_out = open(fa_msa_out, 'w')

for record in SeqIO.parse(fa_msa_in, 'fasta'):
    
    fasta_out.write('>' + record.id + '\n')
    genome_imp_stat = imp_stat.loc[imp_stat['Node'] == record.id]
    beg = genome_imp_stat["FA Beg"].values.copy()
    end = genome_imp_stat["FA End"].values.copy()
    
    ranges2del = []
    for i in range(len(genome_imp_stat)):
        ranges2del.extend(range(beg[i]-1, end[i]))
    ranges2del = list(dict.fromkeys(ranges2del))
    
    print('Genome',record.id,'has',len(ranges2del),'recombinations')
    #print(ranges2del)
    
    print('Number of gaps before',record.seq.count('-'))
    seq = list(record.seq)
    for to_del in ranges2del:
        seq[to_del] = '-'
    seq = ''.join(seq)
    print('Number of gaps after',seq.count('-'))
    print()
    fasta_out.write(seq + '\n')
    
fasta_out.close()

Genome ASM28347v1 has 13510 recombinations
Number of gaps before 16275
Number of gaps after 16380

Genome ASM158613v has 8974 recombinations
Number of gaps before 11526
Number of gaps after 19087

Genome ASM351514v has 5086 recombinations
Number of gaps before 13254
Number of gaps after 16784

Genome ASM351516v has 7455 recombinations
Number of gaps before 12927
Number of gaps after 18969

Genome ASM351518v has 26897 recombinations
Number of gaps before 13473
Number of gaps after 38619

Genome ASM351526v has 26602 recombinations
Number of gaps before 12162
Number of gaps after 37170

Genome ASM351532v has 3923 recombinations
Number of gaps before 12708
Number of gaps after 15198

Genome ASM351538v has 7442 recombinations
Number of gaps before 10695
Number of gaps after 10779

Genome ASM351542v has 1524 recombinations
Number of gaps before 10806
Number of gaps after 10949

Genome ASM351544v has 5126 recombinations
Number of gaps before 11472
Number of gaps after 15185

Genome ASM351548v

# Convert to Phylip

In [403]:
SeqIO.convert(fa_msa_out, "fasta", ph_msa_out, "phylip")

13

# Expand and export the importation_status table

In [404]:
# read Accession Database

with open(accDB, 'rb') as handle:
    acc = pickle.load(handle)
acc.keys()

dict_keys(['ASM28347v1', 'ASM158613v', 'ASM351514v', 'ASM351516v', 'ASM351518v', 'ASM351526v', 'ASM351532v', 'ASM351538v', 'ASM351542v', 'ASM351544v', 'ASM351548v', 'ASM351550v', 'ASM351552v'])

In [405]:
# read Protein Database

prot_db = pd.read_pickle(protDB)

rename_g = {}
for col in prot_db.columns:
    rename_g[col] = col.split('.1_')[1][:10]
    
prot_db = prot_db.rename(columns=rename_g)
prot_db

,ASM351550v,ASM351552v,ASM351518v,ASM351526v,ASM351548v,ASM351542v,ASM351514v,ASM158613v,ASM351516v,ASM28347v1,ASM351532v,ASM351544v,ASM351538v
WP_157771960.1,"{'chr': 'NZ_CP022761.1', 'location': (77, 76, ...",NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP022770.1', 'location': (77, 76, ...","{'chr': 'NZ_CP014702.1', 'location': (77, 76, ...","{'chr': 'NZ_CP022776.1', 'location': (77, 76, ...",NaN,NaN,"{'chr': 'NZ_CP022798.1', 'location': (77, 76, ...",NaN
WP_075462782.1,"{'chr': 'NZ_CP022761.1', 'location': (281, 282...","{'chr': 'NZ_CP022763.1', 'location': (281, 282...",NaN,NaN,"{'chr': 'NZ_CP022757.1', 'location': (281, 282...","{'chr': 'NZ_CP022796.1', 'location': (281, 282...","{'chr': 'NZ_CP022770.1', 'location': (281, 282...","{'chr': 'NZ_CP014702.1', 'location': (281, 282...","{'chr': 'NZ_CP022776.1', 'location': (281, 282...",NaN,"{'chr': 'NZ_CP022774.1', 'location': (281, 282...","{'chr': 'NZ_CP022798.1', 'location': (281, 282...",NaN
WP_013210791.1,"{'chr': 'NZ_CP022761.1', 'location': (2136, 21...","{'chr': 'NZ_CP022763.1', 'location': (2136, 21...","{'chr': 'NZ_CP022788.1', 'location': (2136, 21...","{'chr': 'NZ_CP022759.1', 'location': (2136, 21...","{'chr': 'NZ_CP022757.1', 'location': (2136, 21...","{'chr': 'NZ_CP022796.1', 'location': (2136, 21...","{'chr': 'NZ_CP022770.1', 'location': (2136, 21...","{'chr': 'NZ_CP014702.1', 'location': (2136, 21...","{'chr': 'NZ_CP022776.1', 'location': (2136, 21...","{'chr': 'NC_014311.1', 'location': (2136, 2137...","{'chr': 'NZ_CP022774.1', 'location': (2136, 21...","{'chr': 'NZ_CP022798.1', 'location': (2136, 21...","{'chr': 'NZ_CP023016.1', 'location': (2136, 21..."
WP_075462784.1,"{'chr': 'NZ_CP022761.1', 'location': (3388, 33...","{'chr': 'NZ_CP022763.1', 'location': (3388, 33...",NaN,NaN,"{'chr': 'NZ_CP022757.1', 'location': (3388, 33...","{'chr': 'NZ_CP022796.1', 'location': (3388, 33...","{'chr': 'NZ_CP022770.1', 'location': (3388, 33...","{'chr': 'NZ_CP014702.1', 'location': (3388, 33...","{'chr': 'NZ_CP022776.1', 'location': (3388, 33...",NaN,"{'chr': 'NZ_CP022774.1', 'location': (3388, 33...","{'chr': 'NZ_CP022798.1', 'location': (3388, 33...",NaN
WP_082317214.1,"{'chr': 'NZ_CP022761.1', 'location': (6156, 61...",NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP022770.1', 'location': (6156, 61...","{'chr': 'NZ_CP014702.1', 'location': (6156, 61...","{'chr': 'NZ_CP022776.1', 'location': (6156, 61...",NaN,NaN,"{'chr': 'NZ_CP022798.1', 'location': (6156, 61...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_134928485.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP023017.1', 'location': (2013802,..."
WP_134928414.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP023017.1', 'location': (2021123,..."
WP_134928415.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP023017.1', 'location': (2023468,..."
WP_134928416.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'chr': 'NZ_CP023017.1', 'location': (2028094,..."


for k in acc.keys():
    g2ranges = imp_stat.loc[imp_stat['Node'] == k]
    
    g_beg = g2ranges["FA Beg"].values.copy()
    g_end = g2ranges["FA End"].values.copy()
    
    print(k)
    for i in range(len(g_beg)):
        print(set(acc[k][g_beg[i]:g_end[i]]))

In [406]:
# assign improtations to proteins

g_beg = imp_stat["FA Beg"].values.copy()
g_end = imp_stat["FA End"].values.copy()
g_node = imp_stat["Node"].values.copy()

import_acc = []
import_products = []
for i in range(len(g_beg)):
    import_acc.append(set(acc[g_node[i]][g_beg[i]:g_end[i]]))
    
    products = []
    for a in import_acc[-1]:
        products.append(prot_db.loc[a, g_node[i]]['product'])
    import_products.append(products)
    
imp_stat['Prot-ACC'] = import_acc
imp_stat['Product'] = import_products

In [407]:
imp_stat.to_csv(updated_recombis, index=False)
imp_stat

,Node,Beg,End,FA Beg,FA End,debug_len_must,debug_len_is,Prot-ACC,Product
0,ASM28347v1,203110,203193,77109,77192,83,83,{WP_013209202.1},[[DMT family transporter]]
487,ASM351518v,216232,216310,83231,83309,78,78,{WP_118870158.1},[[glycoside hydrolase family 5 protein]]
570,ASM351526v,216232,216310,83231,83309,78,78,{WP_118870158.1},[[glycoside hydrolase family 5 protein]]
488,ASM351518v,216737,216793,83736,83792,56,56,{WP_118870158.1},[[glycoside hydrolase family 5 protein]]
571,ASM351526v,216737,216793,83736,83792,56,56,{WP_118870158.1},[[glycoside hydrolase family 5 protein]]
...,...,...,...,...,...,...,...,...,...
39,ASM28347v1,6561993,6562052,3202992,3203051,59,59,{WP_172840255.1},[[prepilin peptidase]]
569,ASM351518v,6561996,6562131,3202995,3203130,135,135,{WP_048817055.1},[[prepilin peptidase transmembrane protein]]
652,ASM351526v,6561996,6562131,3202995,3203130,135,135,{WP_048817055.1},[[prepilin peptidase transmembrane protein]]
40,ASM28347v1,6565841,6565897,3204840,3204896,56,56,{WP_173391092.1},[[TonB-dependent receptor]]


In [408]:
imp_stat = pd.read_csv(updated_recombis)
imp_stat = imp_stat.drop(['Node'], axis=1).drop_duplicates()
imp_stat['Product'].value_counts(normalize=True)[:50].to_csv(top_recombis)

# TODO {'WP_071623447.1'} seq length is less than improtation length?
# TODO enrichment with probabilities 
e.g. numOf(T3 effectors with recombinations)/numOf(all T3 effectors in the genome)

In [409]:
#prot_db.loc["WP_071507848.1","ASM1330693"]